<a href="https://colab.research.google.com/github/Waliya451/Fraud-Detection/blob/main/KNN_no_pckg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split


In [9]:
dataset = pd.read_csv("./twitter_human_bots_dataset.csv")
dataset.drop(dataset.columns[0], axis=1, inplace=True)
dataset.drop('created_at', axis=1, inplace=True)
dataset.drop('description', axis=1, inplace=True)
dataset.drop('lang',axis=1, inplace=True)
dataset.drop('profile_background_image_url',axis=1, inplace=True)
dataset.drop('id', axis=1, inplace=True)
dataset.drop('screen_name', axis=1, inplace=True)

In [10]:
df = pd.DataFrame(dataset)
df

,default_profile,default_profile_image,favourites_count,followers_count,friends_count,geo_enabled,location,profile_image_url,statuses_count,verified,average_tweets_per_day,account_age_days,account_type
0,False,False,4,1589,4,False,unknown,http://pbs.twimg.com/profile_images/7874121826...,11041,False,7.870,1403,bot
1,False,False,536,860,880,False,Estados Unidos,http://pbs.twimg.com/profile_images/8023296328...,252,False,0.183,1379,human
2,False,False,3307,172,594,True,"Los Angeles, CA",http://pbs.twimg.com/profile_images/1278890453...,1001,False,0.864,1159,human
3,True,False,8433,517,633,True,"Birmingham, AL",http://pbs.twimg.com/profile_images/1284884924...,1324,False,0.889,1489,human
4,False,False,88,753678,116,True,"England, United Kingdom",http://pbs.twimg.com/profile_images/9952566258...,4202,True,1.339,3138,human
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37433,True,False,651,139,1105,False,Kingston Upon Hull,http://pbs.twimg.com/profile_images/5844684414...,340,False,0.084,4028,human
37434,False,False,8839,1121486,605,True,unknown,http://pbs.twimg.com/profile_images/9511349456...,24970,True,8.976,2782,human
37435,True,False,399,85630,190,False,En constante movimiento,http://pbs.twimg.com/profile_images/1174733822...,6174,True,2.226,2773,human
37436,False,False,967,138,166,True,"Los Angeles, CA",http://pbs.twimg.com/profile_images/1265807908...,982,False,0.339,2899,human


In [11]:
pd.set_option('future.no_silent_downcasting', True)
result = df['account_type'].replace({'human': 0, 'bot': 1})
df['account_type'] = result.infer_objects(copy=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37438 entries, 0 to 37437
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   default_profile         37438 non-null  bool   
 1   default_profile_image   37438 non-null  bool   
 2   favourites_count        37438 non-null  int64  
 3   followers_count         37438 non-null  int64  
 4   friends_count           37438 non-null  int64  
 5   geo_enabled             37438 non-null  bool   
 6   location                37434 non-null  object 
 7   profile_image_url       37437 non-null  object 
 8   statuses_count          37438 non-null  int64  
 9   verified                37438 non-null  bool   
 10  average_tweets_per_day  37438 non-null  float64
 11  account_age_days        37438 non-null  int64  
 12  account_type            37438 non-null  int64  
dtypes: bool(4), float64(1), int64(6), object(2)
memory usage: 2.7+ MB


In [12]:
feats = df.drop('account_type', axis=1)
feats = feats.drop('profile_image_url', axis=1)
feats = feats.drop('location', axis=1).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(feats, df['account_type'].to_numpy(), test_size=0.2, random_state=42)

In [13]:
def knn_predict(X_train, y_train, x_test, k=5):
    distances = [np.sqrt(np.sum((x_test - x_train)**2)) for x_train in X_train]
    k_indices = np.argsort(distances)[:k]
    k_nearest_labels = [y_train[i] for i in k_indices]
    most_common = Counter(k_nearest_labels).most_common(1)
    return most_common[0][0]


In [14]:
def knn_predict_batch(X_train, y_train, X_test, k=5):
    predictions = [knn_predict(X_train, y_train, x_test, k) for x_test in X_test]
    return np.array(predictions)


In [15]:
def accuracy_score(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Predict the classes for the test set
y_pred = knn_predict_batch(X_train, y_train, X_test, k=5)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.8505608974358975
